## Convert input data from S-JTSK to WGS-84

Let's download address points from State Administration of Land Surveying and Cadastre of the Czech Republic http://nahlizenidokn.cuzk.cz/stahniadresnimistaruian.aspx

Data are stored in the form of CVS file in [data/20180430_OB_573108_ADR.csv](data/20180430_OB_573108_ADR.csv)


In [ ]:
with open("data/20180430_OB_573108_ADR.csv") as indata:
    for line in indata.readlines()[0:10]:
        line = line.strip()
        print(line)

Parse the input lines

In [ ]:
data = {}
with open("data/20180430_OB_573108_ADR.csv") as indata:
    for line in indata.readlines()[1:]:
        line = line.strip()
 
        (code_adm, city, name, code_momc, name_momc, code_mop,
         name_mop, code_part, name_part, code_street, name_street,
         type_so, house_number, house_orientation, char, zipcode, Y, X, valid) = line.split(";")
        
        if not Y and not X:
            continue
        
        data[code_adm] = {
            "street": name_street,
            "number": house_number,
            "city": name,
            "zip": zipcode,
            "Y": -1*float(Y),
            "X": -1*float(X)
        }

NOTE: `Y` and `X` coordinates in S-JTSK are `x` and `y` in cartesian system.

NOTE: S-JTSK uses positive values, but EPSG:5514 assumes negative values, therefore `x = -1*Y`

Let's convert `x` and `y` coordinates from czech local [S-JTSK](http://epsg.io/5514) to global [WGS-84](http://epsg.io/4326) system.

### Transformation using PyProj

[PyProj](https://pypi.org/project/pyproj/) is Python bindings to [Proj.4](https://proj4.org/) library.


In [ ]:
from pyproj import Proj, transform
import pyproj

original = Proj("+proj=krovak +lat_0=49.5 +lon_0=24.83333333333333 +alpha=30.28813972222222 +k=0.9999 +x_0=0 +y_0=0 +ellps=bessel +pm=greenwich +units=m +no_defs +towgs84=570.8,85.7,462.8,4.998,1.587,5.261,3.56")

destination = Proj(init='EPSG:4326')

for adm in data:
    x, y = (data[adm]["Y"], data[adm]["X"])
    trans_coords = transform(original, destination, x, y)
    print(trans_coords)
    data[adm]["x"] = trans_coords[0]
    data[adm]["y"] = trans_coords[1]


### Creating new GeoJSON file

Let's create new [GeoJSON](http://geojson.org) vector structure with address points

In [ ]:
import json

features = {
    "type": "FeatureCollection",
    "features":[]
}

for adm in data:
    feature = {
        "type":"Feature",
        "properties": {
            "city": data[adm]["city"],
            "zip": data[adm]["zip"],
            "street": data[adm]["street"],
            "number": data[adm]["number"]
        },
        "geometry": {
            "type": "Point",
            "coordinates": [data[adm]["x"], data[adm]["y"]]
        }
    }
    
    features["features"].append(feature)

print(features["features"][0:5])

Write it to output GeoJSON file

In [ ]:
import json

with open("../outputs/addresses.geojson", "w") as out:
    json.dump(features, out)

And download the data for QGIS: [../outputs/addresses.geojson](../outputs/addresses.geojson)

![](images/addresses.png)

---
[<- Introduction](01_projections.ipynb) | [GeoDjango ->](../06_geodjango/00_Introduction.ipynb)